In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Input, Dense, Lambda, Flatten, Reshape, Layer
from keras.layers import Conv1D, MaxPooling1D, LSTM, UpSampling1D
from keras.models import Model
from keras import backend as K
from keras import metrics
from scipy.stats import norm

# Image Parameters

In [ ]:
# input image dimensions
img_rows, img_cols, img_chns = 28, 28, 1

# number of convolutional filters to use
filters = 64

# convolution kernel size
num_conv = 3

batch_size = 100
if K.image_data_format() == 'channels_first':
    original_img_size = (img_chns, img_rows, img_cols)
else:
    original_img_size = (img_rows, img_cols, img_chns)

# Import MNIST Images

In [ ]:
from keras.datasets import mnist

In [ ]:
(x_train, _), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape((x_train.shape[0],) + original_img_size)
x_test = x_test.astype('float32') / 255.
x_test = x_test.reshape((x_test.shape[0],) + original_img_size)

print('x_train.shape:', x_train.shape)

### Convert to shape of multivariate time series

In [ ]:
x_train = np.asarray([x_train[i,:,:,:].reshape(28*28) for i in range(x_train.shape[0])])
x_test = np.asarray([x_test[i,:,:,:].reshape(28*28) for i in range(x_test.shape[0])])

# Simple LSTM
Without "return sequence" not "stateful" parameter

In [ ]:
model.predict(x_train[0:10].reshape(1,10,784)).shape

# ConvNet1

In [ ]:
x = Input(shape=(10,28*28))
conv_1 = Conv1D(filters=1, 
                kernel_size=2, 
                padding='same', 
                activation='relu')(x)
#max_pooling_1 = MaxPooling1D(pool_size=2)(conv_1)
#lstm_1 = LSTM(100)(max_pooling_1)
model = Model(x,conv_1)
model.compile(optimizer="adam", loss="mse")

In [ ]:
print model.predict(x_train[0:10].reshape(1,10,784)).shape
print x_train[0:10].reshape(1,10,784).shape

In [ ]:
model.summary()

In [ ]:
for layer in model.layers:
    if layer.get_weights():
        print layer.get_weights()[0].shape, layer.get_weights()[1].shape

# ConvNet1D + MaxPooling

In [ ]:
x = Input(shape=(10,28*28))
conv_1 = Conv1D(filters=1, 
                kernel_size=2, 
                padding='same', 
                activation='relu')(x)
max_pooling_1 = MaxPooling1D(pool_size=2)(conv_1)
#lstm_1 = LSTM(100)(max_pooling_1)
model = Model(x,max_pooling_1)
model.compile(optimizer="adam", loss="mse")

# Maxpooling and upsampling

In [ ]:
x = Input(shape=(10,28*28))
max_pooling_1 = MaxPooling1D(pool_size=2)(x)
upsampling_1 = UpSampling1D(size=2)(max_pooling_1)
model = Model(x,upsampling_1)
model.compile(optimizer="adam", loss="mse")

# Auto Encoder

In [ ]:
from keras.layers import RepeatVector

In [ ]:
timesteps = 10
input_dim = 28*28
latent_dim = 100

inputs = Input(shape=(timesteps, input_dim))
encoded = LSTM(latent_dim)(inputs)

decoded = RepeatVector(timesteps)(encoded)
decoded = LSTM(input_dim, return_sequences=True)(decoded)

sequence_autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded)
model.compile(optimizer="adam", loss="mse")

In [ ]:
model.predict(x_train[0:10].reshape(1,10,784)).shape, encoder.predict(x_train[0:10].reshape(1,10,784)).shape

# Deep Auto Encoder

In [ ]:
x = Input(shape=(10,28*28))
lstm_1 = LSTM(32)(x)
model = Model(x,lstm_1)
model.compile(optimizer="adam", loss="mse")

In [ ]:
model.predict(x_train[0:30].reshape(3,10,784)).shape

# Simple stateful LSTM
Without "return sequence"
Stateful LSTM share their internal states between examples

In [ ]:
x = Input(shape=(10,28*28),batch_shape=(3,10,28*28))
lstm_1 = LSTM(32,
              stateful=True)(x)
model = Model(x,lstm_1)
model.compile(optimizer="adam", loss="mse")

# Simple return sequence LSTM 
Without "stateful"

# Deep Auto Encoder with ConvNet1D

In [ ]:
timesteps = 10
input_dim = 28*28
latent_dim = 100

inputs = Input(shape=(timesteps, input_dim))
conv_1 = Conv1D(filters=300, 
                kernel_size=2, 
                padding='same', 
                activation='relu')(inputs)
max_pooling_1 = MaxPooling1D(pool_size=2)(conv_1)
encoded_h1 = LSTM(300, return_sequences=True)(max_pooling_1)

encoded = LSTM(latent_dim)(encoded_h1)
decoded = RepeatVector(timesteps)(encoded)

#upsampling_1 = UpSampling1D(size=2)(decoded)
deconv_1 = Conv1D(filters=300, 
                kernel_size=2, 
                padding='same', 
                activation='relu')(decoded)
decoded_h1 = LSTM(300, return_sequences=True)(deconv_1)
decoded = LSTM(input_dim, return_sequences=True)(decoded_h1)

sequence_autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded)
sequence_autoencoder.compile(optimizer="adam", loss="mse")

In [ ]:
print sequence_autoencoder.predict(x_train[0:10].reshape(1,10,784)).shape
print x_train[0:10].reshape(1,10,784).shape

In [ ]:
sequence_autoencoder.summary()

# Variational Sequential Auto Encoder

## Create sampling function

In [ ]:
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=epsilon_std)
    return z_mean + K.exp(z_log_var) * epsilon

## Create the loss function

In [ ]:
def vae_loss(x, x_decoded_mean_squash):
    x = K.flatten(x)
    x_decoded_mean_squash = K.flatten(x_decoded_mean_squash)
    xent_loss = img_rows * img_cols * metrics.binary_crossentropy(x, x_decoded_mean_squash)
    kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return K.mean(xent_loss + kl_loss)

## Meta Parameters

Meta parameters

In [ ]:
x = Input(shape=(10,28*28))
lstm_2 = LSTM(32, return_sequences=True)(x)
model = Model(x,lstm_2)
model.compile(optimizer="adam", loss="mse")

# Stacked LSTM

In [ ]:
timesteps = 10
input_dim = 28*28
latent_dim = 100

inputs = Input(shape=(timesteps, input_dim))
encoded_h1 = LSTM(300, return_sequences=True)(inputs)
encoded = LSTM(latent_dim)(encoded_h1)

decoded = RepeatVector(timesteps)(encoded)
decoded_h1 = LSTM(300, return_sequences=True)(decoded)
decoded = LSTM(input_dim, return_sequences=True)(decoded_h1)

sequence_autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded)
model.compile(optimizer="adam", loss="mse")

In [ ]:
x = Input(shape=(10,28*28))
lstm_2 = LSTM(32, return_sequences=True)(x)
lstm_3 = LSTM(15, return_sequences=True)(lstm_2)
lstm_4 = LSTM(5)(lstm_3)
model = Model(x,lstm_4)
model.compile(optimizer="adam", loss="mse")